In [ ]:
#sound for alarm has been imported

from IPython.display import Audio
sound_file = 'beep.wav'

In [ ]:
from flask import Flask, render_template, Response
import cv2
import face_recognition
import numpy as np
app=Flask(__name__)
#OpenCV is required to import the `cv2` library to read from your webcam

#parameter 0 in VideoCapture(0) to access webcam
video_capture = cv2.VideoCapture(0)

#A sample picture of the authorised user has been saved in the User folder
#Objective here is to compare the picture saved and the person in the webcam and then show the results
user_image = face_recognition.load_image_file("User/authorised_user.jpg")
user_face_encoding = face_recognition.face_encodings(user_image)[0]

#Create arrays of known face encodings and their names
#More than 1 authorised users can be present
known_face_encodings = [
    user_face_encoding
]
known_face_names = [
    "Aishwarya"
]

#Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

def gen_frames():
    auth = 0 #variable auth has been initialised
    play = 0 #variable auth has been initialised
    while True:
        success, frame = video_capture.read()  # read the camera frame
        if not success:
            break
        else:
            #Resize frame of video to 1/4 size for faster face recognition processing
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            #Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            rgb_small_frame = small_frame[:, :, ::-1]

            #process every other frame of video to save time
           
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                    auth += 1
                else:
                    name = "Unknown"
                    auth -= 1
                if auth < -1 and play == 0:
                    display(Audio(sound_file, autoplay=True))
                    play = 1
                    
                #auth variable has been set to 0, if the face matches the auth value will increment
                #and will display the authorised user's name
                #if there is an unknown person on webcame the auth value will decrement and an alarm
                #will start ringing.
                    
                face_names.append(name)

            # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
@app.route('/')
def home():
    return render_template('index.html')
@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')
if __name__=='__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)